# MAPPO - IPPO algorithms implementation

> In this notebook, we implement two state-of-the-art Multi Agent Reinforcement Leaning (MARL) algorithms **Multi-Agent Proximal Policy Optimization [MAPPO](https://arxiv.org/pdf/2103.01955)** and **Independent Proximal Policy Optimization [IPPO](https://arxiv.org/pdf/2011.09533)** in our environment. 


> Tutorial based on [Multi-Agent Reinforcement Learning (PPO) with TorchRL Tutorial](https://pytorch.org/rl/stable/tutorials/multiagent_ppo.html).

### Simulation overview

> We simulate our environment with an initial population of **20 human agents**. These agents navigate the environment and eventually converge on the fastest path. After this convergence, we will transition **10 of these human agents** into **machine agents**, specifically autonomous vehicles (AVs), which will then employ either the MAPPO or IPPO reinforcement learning algorithms to further refine their learning.

![Alt text](../../docs/img/env.png)


#### Imported libraries

In [1]:
import os
import sys
import torch
from tqdm import tqdm

from tensordict.nn import TensorDictModule
from torchrl.collectors import SyncDataCollector
from torch.distributions import Categorical
from torchrl.envs.libs.pettingzoo import PettingZooWrapper
from torchrl.envs.transforms import TransformedEnv, RewardSum
from torchrl.envs.utils import check_env_specs
from torchrl.data.replay_buffers import ReplayBuffer
from torchrl.data.replay_buffers.samplers import SamplerWithoutReplacement
from torchrl.data.replay_buffers.storages import LazyTensorStorage
from torchrl.modules import MultiAgentMLP, ProbabilisticActor
from torchrl.objectives.value import GAE
from torchrl.objectives import ClipPPOLoss, ValueEstimators


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))

from RouteRL.keychain import Keychain as kc
from RouteRL.environment.environment import TrafficEnvironment
from RouteRL.services.plotter import Plotter
from RouteRL.utilities import get_params

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


#### Hyperparameters setting

In [2]:
# Devices
device = (
    torch.device(0)
    if torch.cuda.is_available()
    else torch.device("cpu")
)

print("device is: ", device)
vmas_device = device  # The device where the simulator is run

# Sampling
frames_per_batch = 40  # Number of team frames collected per training iteration
n_iters = 10  # Number of sampling and training iterations - the episodes the plotter plots
total_frames = frames_per_batch * n_iters

# Training
num_epochs = 100  # Number of optimization steps per training iteration
minibatch_size = 2  # Size of the mini-batches in each optimization step
lr = 3e-4  # Learning rate
max_grad_norm = 1.0  # Maximum norm for the gradients

# PPO
clip_epsilon = 0.2  # clip value for PPO loss
gamma = 0.99  # discount factor
lmbda = 0.9  # lambda for generalised advantage estimation
entropy_eps = 1e-4  # coefficient of the entropy term in the PPO loss

device is:  cpu


#### Environment initialization

> In this example, the environment initially contains only human agents.

In [3]:
params = get_params("params.json")

env = TrafficEnvironment(params[kc.RUNNER], params[kc.ENVIRONMENT], params[kc.SIMULATOR], params[kc.AGENT_GEN], params[kc.AGENTS], params[kc.PLOTTER])

[CONFIRMED] Environment variable exists: SUMO_HOME
[SUCCESS] Added module directory: C:\Program Files (x86)\Eclipse\Sumo\tools


FileNotFoundError: [Errno 2] No such file or directory: '../RouteRL/network_and_config/csomor/paths.csv'

In [4]:
print("Number of total agents is: ", len(env.all_agents), "\n")
print("Agents are: ", env.all_agents, "\n")
print("Number of human agents is: ", len(env.human_agents), "\n")
print("Number of machine agents (autonomous vehicles) is: ", len(env.machine_agents), "\n")

Number of total agents is:  20 

Agents are:  [Human 0, Human 1, Human 2, Human 3, Human 4, Human 5, Human 6, Human 7, Human 8, Human 9, Human 10, Human 11, Human 12, Human 13, Human 14, Human 15, Human 16, Human 17, Human 18, Human 19] 

Number of human agents is:  20 

Number of machine agents (autonomous vehicles) is:  0 



> Reset the environment and the connection with SUMO

In [5]:
env.start()
env.reset()

({}, {})

#### Human learning

In [6]:
num_episodes = 100

for episode in range(num_episodes):
    env.step()

#### Mutation

> **Mutation**: a portion of human agents are converted into machine agents (autonomous vehicles). You can adjust the number of agents to be mutated in the <code style="color:white">/params.json</code> file.

In [7]:
env.mutation()

In [8]:
print("Number of total agents is: ", len(env.all_agents), "\n")
print("Agents are: ", env.all_agents, "\n")
print("Number of human agents is: ", len(env.human_agents), "\n")
print("Number of machine agents (autonomous vehicles) is: ", len(env.machine_agents), "\n")

Number of total agents is:  20 

Agents are:  [Machine 19, Machine 5, Machine 17, Machine 9, Machine 8, Machine 0, Machine 14, Machine 11, Machine 10, Machine 18, Human 1, Human 2, Human 3, Human 4, Human 6, Human 7, Human 12, Human 13, Human 15, Human 16] 

Number of human agents is:  10 

Number of machine agents (autonomous vehicles) is:  10 



> Create a group that contains all the machine (RL) agents.

>  **Hint:** the agents aren't competely independent in this example.

In [9]:
machine_list = []
for machines in env.machine_agents:
    machine_list.append(str(machines.id))
      
group = {'agents': machine_list}

#### PettingZoo environment wrapper

In [10]:
env = PettingZooWrapper(
    env=env,
    use_mask=True,
    categorical_actions=True,
    done_on_any = False,
    group_map=group,
    device=device
)

> The environment is defined by a series of metadata that describe what can be expected during its execution. 

There are four specs to look at:

- <code style="color:white">action_spec</code> defines the action space;

- <code style="color:white">reward_spec</code> defines the reward domain;

- <code style="color:white">done_spec</code> defines the done domain;

- <code style="color:white">observation_spec</code> which defines the domain of all other outputs from environment steps;

In [11]:
print("action_spec:", env.full_action_spec, "\n\n")
print("reward_spec:", env.full_reward_spec, "\n\n")
print("done_spec:", env.full_done_spec, "\n\n")
print("observation_spec:", env.observation_spec, "\n\n")

action_spec: CompositeSpec(
    agents: CompositeSpec(
        action: DiscreteTensorSpec(
            shape=torch.Size([10]),
            space=DiscreteBox(n=3),
            device=cpu,
            dtype=torch.int64,
            domain=discrete), device=cpu, shape=torch.Size([10])), device=cpu, shape=torch.Size([])) 


reward_spec: CompositeSpec(
    agents: CompositeSpec(
        reward: UnboundedContinuousTensorSpec(
            shape=torch.Size([10, 1]),
            space=None,
            device=cpu,
            dtype=torch.float32,
            domain=continuous), device=cpu, shape=torch.Size([10])), device=cpu, shape=torch.Size([])) 


done_spec: CompositeSpec(
    done: DiscreteTensorSpec(
        shape=torch.Size([1]),
        space=DiscreteBox(n=2),
        device=cpu,
        dtype=torch.bool,
        domain=discrete),
    terminated: DiscreteTensorSpec(
        shape=torch.Size([1]),
        space=DiscreteBox(n=2),
        device=cpu,
        dtype=torch.bool,
        domain

> Agent group mapping

In [12]:
print("env.group is: ", env.group_map, "\n\n")

env.group is:  {'agents': ['19', '5', '17', '9', '8', '0', '14', '11', '10', '18']} 




#### Transforms

> We can append any TorchRL transform we need to our environment. These will modify its input/output in some desired way. In multi-agent contexts, it is paramount to provide explicitly the keys to modify.



Here we instatiate a <code style="color:white">RewardSum</code> transformer that will sum rewards over episode.

In [13]:
env = TransformedEnv(
    env,
    RewardSum(in_keys=[env.reward_key], out_keys=[("agents", "episode_reward")]),
)

The <code style="color:white">check_env_specs()</code> function runs a small rollout and compared it output against the environment specs. It will raise an error if the specs aren't properly defined.

In [14]:
check_env_specs(env)


2024-10-08 10:38:45,267 [torchrl][INFO] check_env_specs succeeded!


In [15]:
reset_td = env.reset()

#### Rollout

With <code style="color:white">env.rollout(n_steps)</code> we can get an overview of what the environment inputs and outputs look like.

##### Observation and Action Availability in TorchRL

In the TorchRL framework, the observations are available in both the root and next states, as they can be accessed during both the reset phase and after executing a step.

##### Action Availability
Actions are exclusively available in the root state, as there are no actions generated as a result of a step.

##### Reward Availability
Rewards are provided only in the next state, as no rewards are issued at the reset time.


In [16]:
n_rollout_steps = 5
rollout = env.rollout(n_rollout_steps)
print("rollout of three steps:", rollout, "\n\n\n")
print("Shape of the rollout TensorDict:", rollout.batch_size, "\n\n\n")

rollout of three steps: TensorDict(
    fields={
        agents: TensorDict(
            fields={
                action: Tensor(shape=torch.Size([5, 10]), device=cpu, dtype=torch.int64, is_shared=False),
                done: Tensor(shape=torch.Size([5, 10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                episode_reward: Tensor(shape=torch.Size([5, 10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                mask: Tensor(shape=torch.Size([5, 10]), device=cpu, dtype=torch.bool, is_shared=False),
                observation: Tensor(shape=torch.Size([5, 10, 3]), device=cpu, dtype=torch.float32, is_shared=False),
                terminated: Tensor(shape=torch.Size([5, 10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                truncated: Tensor(shape=torch.Size([5, 10, 1]), device=cpu, dtype=torch.bool, is_shared=False)},
            batch_size=torch.Size([5, 10]),
            device=cpu,
            is_shared=False),
        done: Tensor(sh

#### Policy/Actor network

The Proximal Policy Optimization (PPO) algorithm employs a **stochastic policy** to effectively handle exploration during training. Instead of directly outputting a single action, the neural network generates the parameters of a probability distribution over the action space. Actions are then sampled from this distribution.

For discrete action spaces, the neural network outputs the **logits**, which represent the unnormalized scores for each possible action. These logits are subsequently transformed into a probability distribution using a softmax function. The agent samples an action from this distribution, ensuring a balance between exploration and exploitation during learning.

This stochastic approach is essential for robust exploration, as it prevents the policy from becoming deterministic too early, which could lead to suboptimal solutions in complex environments.

In [17]:
share_parameters_policy = False 

policy_net = torch.nn.Sequential(
    MultiAgentMLP(
        n_agent_inputs = env.observation_spec["agents", "observation"].shape[-1],
        n_agent_outputs = env.action_spec.space.n,
        n_agents = env.n_agents,
        centralised=False,
        share_params=share_parameters_policy,
        device=device,
        depth=3,
        num_cells=64,
        activation_class=torch.nn.Tanh,
    ),
)

> The neural network is wrapped in a `TensorDictModule`, which is responsible for managing the input and output interactions with the tensordict. Specifically, the module reads from the specified `in_keys`, processes the inputs through the neural network, and writes the resulting outputs to the defined `out_keys`. 

In [18]:
policy_module = TensorDictModule(
    policy_net,
    in_keys=[("agents", "observation")],
    out_keys=[("agents", "logits")],
) 

> The `ProbabilisticActor` takes as input the logits which are used to parameterize a probability distribution. Actions are then sampled from this distribution.

In [19]:
policy = ProbabilisticActor(
    module=policy_module,
    spec=env.action_spec,
    in_keys=[("agents", "logits")],
    out_keys=[env.action_key],
    distribution_class=Categorical,
    return_log_prob=True,
    log_prob_key=("agents", "sample_log_prob"),
)

#### Critic network

> The critic reads the observations and returns the corresponding value estimates.

In **Multi-Agent Proximal Policy Optimization (MAPPO)**, the critic is centralized and has access to the full state of the environment, providing **full observability**. This centralized critic enables better coordination between agents by evaluating the global state rather than just individual observations.

In contrast, **Independent Proximal Policy Optimization (IPPO)** uses a **local, decentralized critic**. Similar to the policy, the critic in IPPO is based solely on the agent's local observations, making it more scalable and applicable to fully decentralized environments where agents do not have access to the full global state.


In [20]:
share_parameters_critic = True
mappo = False  # IPPO if False

critic_net = MultiAgentMLP(
    n_agent_inputs=env.observation_spec["agents", "observation"].shape[-1],
    n_agent_outputs=1, 
    n_agents=env.n_agents,
    centralised=mappo,
    share_params=share_parameters_critic,
    device=device,
    depth=4,
    num_cells=64,
    activation_class=torch.nn.ReLU,
)

critic = TensorDictModule(
    module=critic_net,
    in_keys=[("agents", "observation")],
    out_keys=[("agents", "state_value")],
)

> Let's try our policy and critic modules.

In [1]:
print("Running policy:", policy(env.reset()))

NameError: name 'policy' is not defined

In [ ]:
print("Running value:", critic(env.reset()))

#### Collector

Collectors perform the following operations:

1. **Reset Environment**: Initialize the environment.
2. **Compute Action**: Determine the next action using the policy and the latest observation.
3. **Execute Step**: Step through the environment with the computed action.

These operations repeat until the environment signals to stop.

In [21]:
collector = SyncDataCollector(
    env,
    policy,
    device=device,
    storing_device=device,
    frames_per_batch=frames_per_batch,
    total_frames=total_frames,
) 

#### Replay buffer

In an on-policy setting, the replay buffer is refilled each time a batch of data is collected. The data within this buffer is then utilized multiple times over a specified number of epochs.

In [22]:
replay_buffer = ReplayBuffer(
    storage=LazyTensorStorage(
        frames_per_batch, device=device
    ),  
    sampler=SamplerWithoutReplacement(),
    batch_size=minibatch_size,
)

#### PPO loss function

An advantage estimation needs to be computed. An advantage is a value that reflects an expectancy over the return value while dealing with the bias/variance tradeoff. To compute the advantage we need to build the advantage module and pass each batch of data through it before each epoch. 

In [23]:
loss_module = ClipPPOLoss(
    actor_network=policy,
    critic_network=critic,
    clip_epsilon=clip_epsilon,
    entropy_coef=entropy_eps,
    normalize_advantage=False,
)
loss_module.set_keys( 
    reward=env.reward_key,  
    action=env.action_key, 
    sample_log_prob=("agents", "sample_log_prob"),
    value=("agents", "state_value"),
    done=("agents", "done"),
    terminated=("agents", "terminated"),
)

loss_module.make_value_estimator(
    ValueEstimators.GAE, gamma=gamma, lmbda=lmbda
) 

GAE = loss_module.value_estimator

optim = torch.optim.Adam(loss_module.parameters(), lr)

#### Training loop

In [24]:
pbar = tqdm(total=n_iters, desc="episode_reward_mean = 0")

episode_reward_mean_list = []
loss_values = []
loss_entropy = []
loss_objective = []
loss_critic = []

for tensordict_data in collector: ##loops over frame_per_batch

    ## Generate the rollouts
    tensordict_data.set(
        ("next", "agents", "done"),
        tensordict_data.get(("next", "done"))
        .unsqueeze(-1)
        .expand(tensordict_data.get_item_shape(("next", env.reward_key))),  # Adjust index to start from 0
    )
    tensordict_data.set(
        ("next", "agents", "terminated"),
        tensordict_data.get(("next", "terminated"))
        .unsqueeze(-1)
        .expand(tensordict_data.get_item_shape(("next", env.reward_key))),  # Adjust index to start from 0
    )

    # Compute GAE for all agents
    with torch.no_grad():
            GAE(
                tensordict_data,
                params=loss_module.critic_network_params,
                target_params=loss_module.target_critic_network_params,
            )

    data_view = tensordict_data.reshape(-1)  
    replay_buffer.extend(data_view)

    ## Update the policies of the learning agents
    for _ in range(num_epochs):
        for _ in range(frames_per_batch // minibatch_size):
            subdata = replay_buffer.sample()
            loss_vals = loss_module(subdata)

            loss_value = (
                loss_vals["loss_objective"]
                + loss_vals["loss_critic"]
                + loss_vals["loss_entropy"]
            )

            loss_value.backward()

            torch.nn.utils.clip_grad_norm_(
                loss_module.parameters(), max_grad_norm
            ) 

            optim.step()
            optim.zero_grad()

            loss_values.append(loss_value.item())

            loss_entropy.append(loss_vals["loss_entropy"].item())

            loss_objective.append(loss_vals["loss_objective"].item())

            loss_critic.append(loss_vals["loss_critic"].item())


   
    collector.update_policy_weights_()
   
    # Logging
    done = tensordict_data.get(("next", "agents", "done"))  # Get done status for the group

    episode_reward_mean = (
        tensordict_data.get(("next", "agents", "episode_reward"))[done].mean().item()
    )
    episode_reward_mean_list.append(episode_reward_mean)


    pbar.set_description(f"episode_reward_mean = {episode_reward_mean}", refresh=False)
    pbar.update()

episode_reward_mean = -3.485416889190674: 100%|██████████| 10/10 [07:55<00:00, 46.99s/it]

>  Check `\plots` directory to find the plots created from this experiment.

In [25]:
from RouteRL.services import plotter
plotter(params[kc.PLOTTER])

> We can also plot the loss values to get an insight on whether the algorithm converged.

> How can we check if the algorithm converged? 